# First cleaning our medicalknowlege base

In [13]:
import os 
import json
path = os.getcwd()
print(path)
files=os.listdir('medicalKnowledgeBase\Diagnosis_flowchart')
print(files)


c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0
['Acute Coronary Syndrome.json', 'Adrenal Insufficiency.json', 'Alzheimer.json', 'Aortic Dissection.json', 'Asthma.json', 'Atrial Fibrillation.json', 'Cardiomyopathy.json', 'COPD.json', 'Diabetes.json', 'Epilepsy.json', 'Gastro-oesophageal Reflux Disease.json', 'Heart Failure.json', 'Hyperlipidemia.json', 'Hypertension.json', 'Migraine.json', 'Multiple Sclerosis.json', 'Peptic Ulcer Disease.json', 'Pituitary Disease.json', 'Pneumonia.json', 'Pulmonary Embolism.json', 'Stroke.json', 'Thyroid Disease.json', 'Tuberculosis.json', 'Upper Gastrointestinal Bleeding.json']


<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Muhammad Abu Huraira\AppData\Local\Temp\ipykernel_18352\2070653831.py:5: SyntaxWarning: invalid escape sequence '\D'
  files=os.listdir('medicalKnowledgeBase\Diagnosis_flowchart')


In [14]:

folder_path = r"medicalKnowledgeBase/Diagnosis_flowchart"

all_chunks = []

def Convertorfunction(data, parent_key=""):
    items = []
    for key, value in data.items():
        new_key = f"{parent_key}/{key}" if parent_key else key

        if isinstance(value, str):
            items.append({"medicalKB": f"{new_key}: {value}"})

        elif isinstance(value, list):
            items.append({"medicalKB": f"{new_key}: []"})

        elif isinstance(value, dict):
            items.extend(Convertorfunction(value, new_key))

    return items

# this loop is responsible for reading all the files in the folde
for file_name in os.listdir(folder_path):
    if file_name.endswith(".json"):
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        all_chunks.extend(Convertorfunction(data))


# --- Save output ---
with open("ragFile.json", "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, indent=2, ensure_ascii=False)

print("Done! Saved as ragFile.json")


Done! Saved as ragFile.json


In [15]:
import os
import json

# Define paths
base_path = os.getcwd()
folder_path = os.path.join(base_path, "patient_cases")
output_file = os.path.join(base_path, "patient_cases_processed.json")

print(f"Scanning directory: {folder_path}")

processed_cases = []

def process_patient_case(file_path, file_name, disease_group, specific_disease=None):
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {file_path}")
            return None

    inputs = {}
    reasoning = {}
    
    # Separate inputs and reasoning
    for key, value in data.items():
        if key.lower().startswith("input"):
            inputs[key] = value
        else:
            reasoning[key] = value
            
    # Clean inputs
    cleaned_inputs = {}
    # Standardize to input1..input6
    for i in range(1, 7):
        key = f"input{i}"
        val = None
        for k in inputs:
            if k.lower() == key:
                val = inputs[k]
                break
        
        if not val or (isinstance(val, str) and val.strip() == ""):
            val = "NA"
        
        if isinstance(val, str):
            cleaned_inputs[key] = val.strip()
        else:
            cleaned_inputs[key] = val

    case_entry = {
        "file_name": file_name,
        "disease_group": disease_group,
        "specific_disease": specific_disease if specific_disease else "NA",
        "reasoning": reasoning,
        "inputs": cleaned_inputs
    }
    
    return case_entry

# Walk through the directory
if not os.path.exists(folder_path):
    print(f"Error: Directory not found: {folder_path}")
else:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                
                rel_path = os.path.relpath(root, folder_path)
                path_parts = rel_path.split(os.sep)
                
                # path_parts[0] should be 'Finished'
                if len(path_parts) > 1 and path_parts[0] == "Finished":
                    disease_group = path_parts[1]
                    specific_disease = path_parts[2] if len(path_parts) > 2 else None
                    
                    case_data = process_patient_case(file_path, file, disease_group, specific_disease)
                    if case_data:
                        processed_cases.append(case_data)

    # Save the processed data
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(processed_cases, f, indent=2, ensure_ascii=False)

    print(f"Processed {len(processed_cases)} cases. Saved to {output_file}")


Scanning directory: c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\patient_cases
Processed 511 cases. Saved to c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\patient_cases_processed.json
Processed 511 cases. Saved to c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\patient_cases_processed.json


# Combined Output: Knowledge Base + Patient Cases
This cell creates a single JSON file with:
- Upper part: Medical Knowledge Base entries
- Lower part: Patient case studies with reasoning flattened to a single string

In [16]:
import os
import json
import re

# Define paths
base_path = os.getcwd()
kb_folder_path = os.path.join(base_path, "medicalKnowledgeBase", "Diagnosis_flowchart")
patient_cases_folder = os.path.join(base_path, "Finished")
output_file = os.path.join(base_path, "combined_rag_data.json")

print(f"Knowledge Base Path: {kb_folder_path}")
print(f"Patient Cases Path: {patient_cases_folder}")

# Function to clean anonymization placeholders (___) from text
def clean_text(text):
    if not isinstance(text, str):
        return text
    # Remove standalone ___ (with optional surrounding spaces)
    cleaned = re.sub(r'\s*___\s*', ' ', text)
    # Clean up multiple spaces
    cleaned = re.sub(r' +', ' ', cleaned)
    # Clean up spaces before punctuation
    cleaned = re.sub(r' +([,.:;])', r'\1', cleaned)
    return cleaned.strip()

# Counter for generating unique IDs
kb_id_counter = 0

def Convertorfunction(data, parent_key=""):
    global kb_id_counter
    items = []
    for key, value in data.items():
        new_key = f"{parent_key}/{key}" if parent_key else key

        if isinstance(value, str):
            kb_id_counter += 1
            items.append({
                "id": f"KB_{kb_id_counter:04d}",
                "medicalKB": f"{new_key}: {value}"
            })
        elif isinstance(value, list):
            kb_id_counter += 1
            items.append({
                "id": f"KB_{kb_id_counter:04d}",
                "medicalKB": f"{new_key}: []"
            })
        elif isinstance(value, dict):
            items.extend(Convertorfunction(value, new_key))
    return items

knowledge_base_entries = []

if os.path.exists(kb_folder_path):
    for file_name in os.listdir(kb_folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(kb_folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            knowledge_base_entries.extend(Convertorfunction(data))
    print(f"Processed {len(knowledge_base_entries)} knowledge base entries")
else:
    print(f"Warning: Knowledge base folder not found at {kb_folder_path}")

def flatten_reasoning(obj, prefix=""):
    parts = []
    if isinstance(obj, dict):
        for key, value in obj.items():
            new_prefix = f"{prefix} -> {key}" if prefix else key
            if isinstance(value, dict) and value:  # Non-empty dict
                parts.append(flatten_reasoning(value, new_prefix))
            else:
                parts.append(new_prefix)
    return " | ".join(filter(None, parts))

def process_patient_case(file_path, file_name, disease_group, specific_disease=None):
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {file_path}")
            return None

    inputs = {}
    reasoning = {}
    for key, value in data.items():
        if key.lower().startswith("input"):
            inputs[key] = value
        else:
            reasoning[key] = value

    cleaned_inputs = {}
    for i in range(1, 7):
        key = f"input{i}"
        val = None
        for k in inputs:
            if k.lower() == key:
                val = inputs[k]
                break
        
        if not val or (isinstance(val, str) and val.strip() == ""):
            val = "NA"
        
        if isinstance(val, str):
            # Clean the ___ placeholders from input text
            cleaned_inputs[key] = clean_text(val.strip())
        else:
            cleaned_inputs[key] = val

    reasoning_string = flatten_reasoning(reasoning)
    
    # Use filename (without .json extension) as the ID
    case_id = file_name.replace(".json", "")
    
    case_entry = {
        "id": case_id,
        "patient_case": {
            "disease_group": disease_group,
            "specific_disease": specific_disease if specific_disease else "NA",
            "reasoning": reasoning_string,
            "inputs": cleaned_inputs
        }
    }
    
    return case_entry

patient_cases_entries = []

if os.path.exists(patient_cases_folder):
    for root, dirs, files in os.walk(patient_cases_folder):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                
                rel_path = os.path.relpath(root, patient_cases_folder)
                path_parts = rel_path.split(os.sep)
                
                # Get disease group and specific disease from folder structure
                if len(path_parts) >= 1 and path_parts[0] != ".":
                    disease_group = path_parts[0]
                    specific_disease = path_parts[1] if len(path_parts) > 1 else None
                    
                    case_data = process_patient_case(file_path, file, disease_group, specific_disease)
                    if case_data:
                        patient_cases_entries.append(case_data)
    print(f"Processed {len(patient_cases_entries)} patient cases")
else:
    print(f"Warning: Patient cases folder not found at {patient_cases_folder}")

# Combine knowledge base entries (upper part) with patient cases (lower part)
combined_data = knowledge_base_entries + patient_cases_entries

# Save the combined output
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, indent=2, ensure_ascii=False)

print(f"\n=== Summary ===")
print(f"Total Knowledge Base entries: {len(knowledge_base_entries)} (IDs: KB_0001 to KB_{kb_id_counter:04d})")
print(f"Total Patient Cases: {len(patient_cases_entries)}")
print(f"Combined entries: {len(combined_data)}")
print(f"Saved to: {output_file}")

Knowledge Base Path: c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\medicalKnowledgeBase\Diagnosis_flowchart
Patient Cases Path: c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\Finished
Processed 192 knowledge base entries
Processed 511 patient cases

=== Summary ===
Total Knowledge Base entries: 192 (IDs: KB_0001 to KB_0192)
Total Patient Cases: 511
Combined entries: 703
Saved to: c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\combined_rag_data.json
Processed 511 patient cases

=== Summary ===
Total Knowledge Base entries: 192 (IDs: KB_0001 to KB_0192)
Total Patient Cases: 511
Combined entries: 703
Saved to: c:\Users\Muhammad Abu Huraira\Documents\Assignments and Submissions\Semester 7\NLP\A04\NLP_Project04_RAG\mimic-iv-ext-direct-1.0.0\c